In [ ]:
!pip install --upgrade pip
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib

In [ ]:
train_dataset='/home/recallfun/webapps/foodblog/fine-tune-llm/alpaca_finetune_dataset.jsonl'

In [ ]:
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from trl import SFTTrainer

In [ ]:
train_dataset = load_dataset('json', data_files=train_dataset , split='train')

In [ ]:
import matplotlib.pyplot as plt

def plot_data_lengths(tokenized_train_dataset):
    lengths = [len(x['input_ids']) for x in tokenized_train_dataset]
    print(len(lengths))
    print(max(lengths))

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=20, alpha=0.7, color='blue')
    plt.xlabel('Length of input_ids')
    plt.ylabel('Frequency')
    plt.title('Distribution of Lengths of input_ids')
    plt.show()

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from accelerate import Accelerator

current_device = Accelerator().process_index

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "/home/recallfun/llm/models/mistral-7b"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token
train_dataset = train_dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)
model = AutoModelForCausalLM.from_pretrained(model_name,
  device_map="auto",
  quantization_config=bnb_config)

plot_data_lengths(train_dataset)

In [ ]:
plot_data_lengths(train_dataset)

In [ ]:
from peft import prepare_model_for_kbit_training

train_dataset = train_dataset.map(lambda samples: tokenizer(samples["text"],truncation=True,
        max_length=2000,
        padding="max_length",), batched=True)
plot_data_lengths(train_dataset)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32, 
    lora_alpha=64, 
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "w1",
        "w2",
        "w3",
        "lm_head",
    ],
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, config)
print_trainable_parameters(peft_model)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        learning_rate=2.5e-5,
        fp16=True,
        logging_steps=1,
        output_dir="/home/recallfun/llm/finetune/food-blog-mistral-7b-checkpoints",
        max_steps=1000,
        save_total_limit = 2,
save_steps = 250,
load_best_model_at_end=False,
num_train_epochs=3,
optim="paged_adamw_8bit"
    )

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=2000,  # You can specify the maximum sequence length here
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
)
peft_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

# Start the training process
trainer.train()

# Save the fine-tuned model
trainer.model.save_pretrained("/home/recallfun/llm/finetune/food-blog-mistral-7b-finetune")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
text = "You are a helpful assistant whose role is to create posts for the food blog. I will give you the dish name and you will provide the blog data in JSON format.The json format is [{ \"type\": \"title\", \"sequence\": 1, \"text\":\"\",\"formatting\":{} },{ \"type\": \"cuisine\", \"sequence\": 2, \"text\":\"\",\"formatting\":{} },{ \"type\": \"preparationTime\", \"sequence\": 3, \"text\":\"\",\"formatting\":{} },{ \"type\": \"cookingTime\", \"sequence\": 4, \"text\":\"\",\"formatting\":{} },{ \"type\": \"equipmentList\", \"sequence\": 5, \"text\": [],\"formatting\":{} },{ \"type\": \"ingredientList\", \"sequence\": 6, \"text\": [],\"formatting\":{} },{ \"type\": \"instructions\", \"sequence\": 7, \"text\": [],\"formatting\":{}}]. The ingredientList is an array of text. The instructions are also array of text.The elements of the instructions array represent the steps in ascending order. equipmentList is an array of text where each text is the name of the equipment needed to prepare the dish. Each object of the json array will have formatting object with the schema  {\n    \"bold\": true,\n    \"italic\": false,\n    \"fontSize\": \"16px\",\n    \"fontColor\": \"#333333\"\n  }. The format object of the type title should have following values {\"bold\": true, \"italic\": false, \"fontSize\": \"16px\", \"fontColor\": \"#cc0000\"}. The format object of the type cusinie have following values {\"bold\": false, \"italic\": false, \"fontSize\": \"18px\", \"fontColor\": \"#ffcc00\"}. The format object of the type equipmentList have following values {\"bold\": false, \"italic\": true, \"fontSize\": \"14px\", \"fontColor\": \"#990000\"}. The format object of the type ingredientList have following values {\"bold\": false, \"italic\": false, \"fontSize\": \"14px\", \"fontColor\": \"#cc6600\"}.The format object of the type instructions have following values {\"bold\": false, \"italic\": true, \"fontSize\": \"14px\", \"fontColor\": \"#ff3300\"}.  The rest of the json objects will have the default value in the formatting object {\"bold\": false, \"italic\": false, \"fontSize\": \"16px\", \"fontColor\": \"#ff9900\"}.  The name of the dish is \"Peking Duck\".Your task is to add the content in the field \"text\" of every object."
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))